<a href="https://colab.research.google.com/github/mayukistarry/kaggle/blob/main/kaggle1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
1+1

2

# URL

- https://www.kaggle.com/competitions/playground-series-s3e19/overview

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
le = LabelEncoder()

In [ ]:
train['store'] = le.fit_transform(train['store'])
train['country'] = le.fit_transform(train['country'])
train['product'] = le.fit_transform(train['product'])

In [ ]:
test['store'] = le.fit_transform(test['store'])
test['country'] = le.fit_transform(test['country'])
test['product'] = le.fit_transform(test['product'])

In [ ]:
df_train, df_val =train_test_split(train, test_size=0.2)

train_x = df_train[['store', 'country', 'product']]
train_y = df_train[['num_sold']]

val_x = df_val[['store', 'country', 'product']]
val_y = df_val[['num_sold']]

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

In [ ]:
model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13
[LightGBM] [Info] Number of data points in the train set: 109560, number of used features: 3
[LightGBM] [Info] Start training from score 165.923740
[1]	valid_0's l1: 122.468
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 110.588
[3]	valid_0's l1: 99.9185
[4]	valid_0's l1: 90.3636
[5]	valid_0's l1: 81.8151
[6]	valid_0's l1: 74.2076
[7]	valid_0's l1: 67.4312
[8]	valid_0's l1: 61.4542
[9]	valid_0's l1: 56.1821
[10]	valid_0's l1: 51.5434
[11]	valid_0's l1: 47.493
[12]	valid_0's l1: 43.9567
[13]	valid_0's l1: 40.9089
[14]	valid_0's l1: 38.2845
[15]	valid_0's l1: 36.0257
[16]	valid_0's l1: 34.0882
[17]	valid_0's l1: 32.4398
[18]	valid_0's l1: 31.0452
[19]	valid_0's l1: 29.8542
[20]	valid_0's l1: 28.8252

In [ ]:
df_test = pd.read_csv('test.csv')
df_test['store'] = le.fit_transform(df_test['store'])
df_test['country'] = le.fit_transform(df_test['country'])
df_test['product'] = le.fit_transform(df_test['product'])
predict = model.predict(df_test[['store', 'country', 'product']])
df_test['num_sold'] = predict
df_test[['id', 'num_sold']].to_csv("submission.csv", index=False)

In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
df_test[['store', 'country', 'product']].head(20)

,store,country,product
0,Kaggle Learn,Argentina,Using LLMs to Improve Your Coding
1,Kaggle Learn,Argentina,Using LLMs to Train More LLMs
2,Kaggle Learn,Argentina,Using LLMs to Win Friends and Influence People
3,Kaggle Learn,Argentina,Using LLMs to Win More Kaggle Competitions
4,Kaggle Learn,Argentina,Using LLMs to Write Better
5,Kaggle Store,Argentina,Using LLMs to Improve Your Coding
6,Kaggle Store,Argentina,Using LLMs to Train More LLMs
7,Kaggle Store,Argentina,Using LLMs to Win Friends and Influence People
8,Kaggle Store,Argentina,Using LLMs to Win More Kaggle Competitions
9,Kaggle Store,Argentina,Using LLMs to Write Better
